In [1]:
import imageio
import glob
import os

If you are using whole images:

In [9]:
# path to the tifs (also works with jpegs)
tif_dir = os.path.join('C:\\','Users','Cate','RnD','Animations','imagery','geotiff')

# get list of images (note that the filenames should be dated so they create a chronological list, otherwise they will 
# need sorting)
filenames = glob.glob(os.path.join(tif_dir,'*.tif'))
images = []

# make gif using whole image
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('exeter_timelapse.gif', images)

If you want to subset the image with a shapefile first:

In [14]:
import fiona
import rasterio
import rasterio.mask

shapefile_path = os.path.join('C:\\','Users','Cate','RnD','Animations','subset.shp')


# open the shapefile
with fiona.open(shapefile_path, "r") as shapefile:
    features = [feature["geometry"] for feature in shapefile]

# open the original tif
for image in glob.glob(os.path.join(tif_dir,'*.tif')):
    
    image_filename = image[-21:-4]
    out_filename = image_filename + '_subset.tif'
    
    # get subset
    with rasterio.open(image) as src:
        out_image, out_transform = rasterio.mask.mask(src, features,
                                                        crop=True)
        out_meta = src.meta.copy()
        out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
        
    # write new geotiff
    with rasterio.open(os.path.join(tif_dir, 'subsets', out_filename), 'w', **out_meta) as dest:
        dest.write(out_image)
        
# path to the tifs (also works with jpegs)
tif_dir = os.path.join('C:\\','Users','Cate','RnD','Animations','imagery','geotiff')

# get list of images
filenames = glob.glob(os.path.join(tif_dir, 'subsets', '*.tif'))
images = []

# make gif using whole image
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('exeter_timelapse2.gif', images)